In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_cacao = pd.read_csv('flavors_of_cacao.csv', encoding='utf-8')

In [ ]:
df_cacao.head()

In [ ]:
#create a copy
cdf_cacao = df_cacao.copy()

In [ ]:
columns = cdf_cacao.columns

In [ ]:
columns

In [ ]:
#change column names - did NOT work for the first column...
cdf_cacao.rename(columns = {'Company \n(Maker-if known)' : 'Maker' , 'Specific Bean Origin\nor Bar Name' : 'Name' , 'Review\nDate' : 'Review Date' , 'Cocoa\nPercent' : 'Cacao %' , 
                 'Company\nLocation' : 'Made in' , 'Bean\nType' : 'Bean Type' , 'Broad Bean\nOrigin': "Bean Origin"} )

In [ ]:
#different method
cdf_cacao.columns = ['Maker' , 'Name' , 'REF#' , 'Review Date' ,  'Cacao %' , 'Made in' , 'Rating' , 'Bean Type' ,  'Bean Origin' ]
cdf_cacao

In [ ]:
missing_values = (cdf_cacao.isna().sum() / cdf_cacao.shape[0]) * 100
missing_values

In [ ]:
# check if bars have only bean reviewed once
len(df_cacao['Specific Bean Origin\nor Bar Name'].unique())

In [ ]:
dups = cdf_cacao.duplicated()
dups.sum()

In [ ]:
cdf_cacao[['Name', 'Review Date']]

In [ ]:
#are the duplicates because of reviews in different years or a different cacao %?
cdf_cacao.groupby(['Name', 'Review Date', 'Cacao %'])['REF#'].count()

In [ ]:
cdf_cacao

In [ ]:
missing_values = (cdf_cacao.isna().sum() / cdf_cacao.shape[0]) * 100
missing_values

In [ ]:
miss_cols = ['Bean Type', 'Bean Origin']

In [ ]:
unknown_cond = {col:'Unknown' for col in miss_cols}
unknown_cond

In [ ]:
cdf2 = cdf_cacao.fillna(unknown_cond)

In [ ]:
cdf2

In [ ]:
len(cdf_cacao['Bean Type'][1])

In [ ]:
cdf2_cacao = cdf_cacao.replace(u'\xa0', 'Unknown')

In [ ]:
cdf_cacao['Bean Origin']

In [ ]:
cdf2_cacao['Bean Origin'].unique()

# 
##Next step is to identify all blends to limit the number of different types

In [ ]:
cdf3_cacao = cdf2_cacao.astype(str)

In [ ]:
blend = [',', '&', '/']
blends_list = []
for origin in cdf3_cacao['Bean Origin']:
    if any(x in origin for x in blend):
    #',' in origin or "&" in origin or '(' in origin:
         blends_list.append(origin)
        

In [ ]:
blends_list

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Origin'].str.contains(','), 'Bean Origin'] = 'blend'

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Origin'].str.contains('/'), 'Bean Origin'] = 'blend'

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Origin'].str.contains('&'), 'Bean Origin'] = 'blend'

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Origin'].str.contains('and'), 'Bean Origin'] = 'blend'

In [ ]:
origins = cdf3_cacao['Bean Origin'].unique()
origins

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Origin'].str.contains('nan'), 'Bean Origin'] = 'Unknown'

# 
##finding missspelled origins

In [ ]:
origins

In [ ]:
sorigins = sorted(origins)

In [ ]:
sorigins

In [ ]:
cdf3_cacao['Bean Origin'] = cdf3_cacao['Bean Origin'].replace('Domincan Republic', 'Dominican Republic')

In [ ]:
cdf3_cacao['Bean Origin'].unique()

In [ ]:
cdf3_cacao['Bean Type'].unique()

# 
##simplify types


In [ ]:
#start = mystring.find( '(' )
Bean_Type_Simplified = []

for i in cdf3_cacao['Bean Type']:
    start = i.find( '(' )
    if start != -1 :
        result = i[:start-2]
        Bean_Type_Simplified.append(result)
    else:
        Bean_Type_Simplified.append(i)
        
len(Bean_Type_Simplified)

In [ ]:
cdf3_cacao['Bean Type 2'] = Bean_Type_Simplified

In [ ]:
cdf3_cacao['Bean Type 2'].unique()

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Type 2'].str.contains(','), 'Bean Type 2'] = 'Blend'

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Type 2'].str.contains('nan'), 'Bean Type 2'] = 'Unknown'

In [ ]:
cdf3_cacao['Bean Type 2'].unique()

In [ ]:
cdf3_cacao.loc[cdf3_cacao['Bean Type 2'].str.contains('mix'), 'Bean Type 2'] = 'Blend'

In [ ]:
unique_types  = cdf3_cacao['Bean Type 2'].unique()

In [ ]:
unique_types_list = list(unique_types)

In [ ]:
unique_types_list

In [ ]:
results = [[name, [], 0] for name in unique_types]

for (i, element) in enumerate(unique_types):
    for (j, choice) in enumerate(unique_types[i+1:]):
        if fuzz.ratio(element, choice) >= 90:
            results[i][2] += 1
            results[i][1].append(choice)
            results[j+i+1][2] += 1
            results[j+i+1][1].append(element)

duped_types= pd.DataFrame(results, columns=['name', 'duplicates', 'duplicate_count'])

In [ ]:
duped_types

In [ ]:
duped_types2 = duped_types.loc[duped_types['duplicate_count'] == 1]

In [ ]:
duped_types2

# 
##choose items to keep /change and replace them in the dataframe

In [ ]:
duped_types2.iloc[0,1]

In [ ]:

cdf3_cacao['Bean Type 2'] = cdf3_cacao['Bean Type 2'].replace(['Crioll', 'Trinitari','Foraste','Foraster', 'Naciona'],['Criollo','Trinitario','Forastero','Forastero', 'Nacional'])

In [ ]:
cdf3_cacao['Bean Type 2'].unique()

In [ ]:
cdf3_cacao

# 
##get records for highest and lowest ratings

In [ ]:
cdf3_cacao['Rating'] = cdf3_cacao['Rating'].astype(np.float64)

In [ ]:
cdf3_best = cdf3_cacao.loc[cdf3_cacao['Rating'] >=4.5]

In [ ]:
cdf3_best

In [ ]:
cdf3_best = cdf3_cacao.loc[cdf3_cacao['Rating'] >=4.0]

In [ ]:
cdf3_best

In [ ]:
cdf3_worst = cdf3_cacao.loc[cdf3_cacao['Rating'] <=2.0]

In [ ]:
cdf3_worst

# 
##further clean up 

In [ ]:
cdf3_cacao=cdf3_cacao.rename(columns = {'Bean Type 2' : 'Bean Type Group'})

In [ ]:
#cdf3_cacao = cdf3_cacao['Rating'].astype(np.float64)
#cdf3_cacao['Cacao %'] = float(cdf3_cacao["Cacao %"].rstrip("%"))
cdf3_cacao['Cacao %'] = cdf3_cacao['Cacao %'].str.rstrip('%').astype('float64')

In [ ]:
df_ready = cdf3_cacao.reset_index(drop=True).copy()

In [ ]:
df_ready.to_csv('cacao_ready.csv', index=False)

In [ ]:
df_ready